In [1]:
using SymPy
using LinearAlgebra

## 时空度规约定

$$g_{\mu\upsilon}=\mathrm{det}(-1,1,1,1)\\ g^{\mu\upsilon}=\mathrm{det}(-1,1,1,1)\\ \delta^\mu_\upsilon=\det (1,1,1,1)$$

In [2]:
δ = sympy.eye(4); 

g = sympy.diag(-1, 1, 1, 1);
gi = inv(g);

## 自旋矩阵

自旋矩阵$L^\Sigma_{\ \ \Omega}\in \mathrm{SU}(2)$:

$$L^\dagger L=I,\quad \det L=1$$


In [3]:
@vars a b c d

L=[a b; c d];

条件1： $ \det L=1$

In [4]:
cond1 = solve(det(L)-1,[a*d]);
cond1 = a*d=>cond1[1]

a*d => b*c + 1

条件2：$L^\dagger L=I$

In [5]:
# L的逆矩阵（带入条件1）
Li= L |> inv .|> (expr->expr(cond1));

# L的共轭转置
Lc = L.T  .|> conj;

eqs1 = Li-Lc;
eqs1 = [eqs1[i] for i in 1:4];
cond2 = solve(eqs1,[c,d])[1];
cond2 = c=>cond2[1],d=>cond2[2]

(c => -conjugate(b), d => conjugate(a))

## 期待中的旋量度规

期待旋量度规具有提升下降旋量指标的功效

$$\xi_A=\xi^B\epsilon_{BA},\quad \xi^A=\epsilon^{AB}\xi_B$$


对$\epsilon_{\Sigma\Omega}$则暂不作假设，有四个待定分量:

In [6]:
@vars α β γ ζ

ϵ=[α β; γ ζ];

待解方程：

$$\epsilon_{AB} L^A_{\ \ C}L^B_{\ \ D}= \epsilon_{CD}$$

In [7]:
eqs=[sum(ϵ[A,B]*L[A,C]*L[B,D] for A in 1:2, B in 1:2)-ϵ[C,D] for C in 1:2, D in 1:2];
eqs = eqs .|> (expr->expr(cond1)) .|> expand .|> (expr->expr(cond2[1],cond2[2])) .|> simplify;
eqs = [eqs[i] for i in 1:4]

4-element Array{Sym,1}:
        a^2*α - a*β*conjugate(b) - a*γ*conjugate(b) - α + ζ*conjugate(b)^2
 a*b*α - b*β*conjugate(b) - b*γ*conjugate(b) - ζ*conjugate(a)*conjugate(b)
 a*b*α - b*β*conjugate(b) - b*γ*conjugate(b) - ζ*conjugate(a)*conjugate(b)
        b^2*α + b*β*conjugate(a) + b*γ*conjugate(a) + ζ*conjugate(a)^2 - ζ

确定旋量度规分量（解方程）

In [8]:
rs = solve(eqs,[α,β,γ,ζ])

Dict{Any,Any} with 3 entries:
  ζ => 0
  β => -γ
  α => 0

##  确定后的旋量度规


$$\epsilon_{\Sigma\Omega}=\epsilon^{\Sigma\Omega}=\begin{pmatrix} 0 & 1 \\  -1& 0 \end{pmatrix}\\ \bar{\epsilon}_{\Sigma'\Omega'}=\bar{\epsilon}^{\Sigma'\Omega'}=\begin{pmatrix} 0 & 1 \\  -1 & 0 \end{pmatrix}$$

In [9]:
ϵ = sympy.Matrix([0 1; -1 0])

2×2 Array{Sym,2}:
  0  1
 -1  0

## 变换的行列式

$$\det(L)=\frac{1}{2}\epsilon_{AB}\epsilon^{CD}L^A_{\ \ C}L^B_{\ \ D}$$

In [10]:
[sum(ϵ[A,B]*ϵ[C,D]*L[A,C]*L[B,D] for A in 1:2, B in 1:2, C in 1:2, D in 1:2)]

1-element Array{Sym,1}:
 2*a*d - 2*b*c

## 旋量度规的性质

旋量度规的基旋量表示:

$$\epsilon^{AB}=\omicron^A\iota^B-\iota^A\omicron^B\\ \epsilon_{AB}=\omicron_A\iota_B-\iota_A\omicron_B$$

性质1：

$$\epsilon^A_{\ \ C}=\epsilon^{AB}\epsilon_{BC}=-\delta^A_C =\omicron^A\iota_C-\iota^A\omicron_C\\ \epsilon^\Sigma_{\ \ \Omega}=\begin{pmatrix} 0 & 1 \\ -1 & 0 \end{pmatrix}$$


In [11]:
[sum(ϵ[A,B]*ϵ[B,C] for B in 1:2) for A in 1:2, C in 1:2] # |> println

2×2 Array{Sym,2}:
 -1   0
  0  -1

性质2：

$$\epsilon_{AB}=\epsilon^{CD}\epsilon_{CA}\epsilon_{DB}$$

【计算验证后，务必再用基底表示手工验证】


In [12]:
[ϵ[A,B]-sum(ϵ[C,D]*ϵ[C,A]*ϵ[D,B] for C in 1:2, D in 1:2) for A in 1:2, B in 1:2]

2×2 Array{Sym,2}:
 0  0
 0  0

性质1'：

$\bar{\epsilon}^{A'B'}\bar{\epsilon}_{B'C'}=-\delta^A_C$

同样可验证

In [13]:
[sum(ϵ[A1,B1]*ϵ[B1,C1] for B1 in 1:2)+δ[A1,C1] for A1 in 1:2, C1 in 1:2]

2×2 Array{Sym,2}:
 0  0
 0  0

性质2'：

$\bar{\epsilon}_{A'B'}=\bar{\epsilon}^{C'D'}\bar{\epsilon}_{C'A'}\bar{\epsilon}_{D'B'}$

In [14]:
[ϵ[A1,B1]-sum(ϵ[C1,D1]ϵ[C1,A1]ϵ[D1,B1] for C1 in 1:2, D1 in 1:2) for A1 in 1:2, B1 in 1:2]

2×2 Array{Sym,2}:
 0  0
 0  0

## 泡利矩阵

In [15]:
σ_0 = sympy.Matrix([1 0;0 1])*sqrt(1//2);
σ_1 = sympy.Matrix([0 1;1 0])*sqrt(1//2);
σ_2 = sympy.Matrix([0 -IM;IM 0])*sqrt(1//2);
σ_3 = sympy.Matrix([1 0;0 -1])*sqrt(1//2);
σ=[σ_0,σ_1,σ_2,σ_3];
σ=[σ[i][j,k] for i in 1:4,j in 1:2 ,k in 1:2];
[σ[i,j,k] for j in 1:2 ,k in 1:2,  i in 1:4]

2×2×4 Array{Sym,3}:
[:, :, 1] =
 0.707106781186548                  0
                 0  0.707106781186548

[:, :, 2] =
                 0  0.707106781186548
 0.707106781186548                  0

[:, :, 3] =
                   0  -0.707106781186548*I
 0.707106781186548*I                     0

[:, :, 4] =
 0.707106781186548                   0
                 0  -0.707106781186548

$\sigma^\mu_{\ \ AA'}=g^{\mu\upsilon}\sigma_\upsilon^{\ \ BB'}\epsilon_{BA}\bar{\epsilon}_{B'A'}$

In [16]:
σi=[sum(gi[μ,υ]*σ[υ,B,B1]*ϵ[B,A]*ϵ[B1,A1] for υ in 1:4, B in 1:2, B1 in 1:2) 
    for μ in 1:4, A in 1:2 , A1 in 1:2];
[sum(g[μ,υ]*σi[υ,A,B] for υ in 1:4) for A in 1:2 ,B in 1:2, μ in 1:4]

2×2×4 Array{Sym,3}:
[:, :, 1] =
 0.707106781186548                  0
                 0  0.707106781186548

[:, :, 2] =
                  0  -0.707106781186548
 -0.707106781186548                   0

[:, :, 3] =
                   0  -0.707106781186548*I
 0.707106781186548*I                     0

[:, :, 4] =
 -0.707106781186548                  0
                  0  0.707106781186548

In [17]:
[sum(σ[μ,A,A1]*σi[υ,A,A1] for A in 1:2, A1 in 1:2) for υ in 1:4, μ in 1:4]

4×4 Array{Sym,2}:
 -1.00000000000000                  0                  0                  0
                 0  -1.00000000000000                  0                  0
                 0                  0  -1.00000000000000                  0
                 0                  0                  0  -1.00000000000000

In [18]:
[sum(σ[μ,A,A1]*σi[μ,B,B1] for μ in 1:4)+δ[A,B]*δ[A1,B1] for A in 1:2, A1 in 1:2, B in 1:2, B1 in 1:2 ]

2×2×2×2 Array{Sym,4}:
[:, :, 1, 1] =
 -2.22044604925031e-16  0
                     0  0

[:, :, 2, 1] =
                     0  0
 -2.22044604925031e-16  0

[:, :, 1, 2] =
 0  -2.22044604925031e-16
 0                      0

[:, :, 2, 2] =
 0                      0
 0  -2.22044604925031e-16

$g^{\mu\upsilon}\sigma_\mu^{\ \ AA'}\sigma_\upsilon^{BB'}-\epsilon^{AB}\epsilon^{\ \ A'B'}=0$

这个直接验证很麻烦

In [19]:
[sum(gi[μ,υ]*σ[μ,A,A1]*σ[υ,B,B1] for μ in 1:4, υ in 1:4)+ϵ[A,B]*ϵ[A1,B1]
        for A in 1:2, B in 1:2, A1 in 1:2, B1 in 1:2]

2×2×2×2 Array{Sym,4}:
[:, :, 1, 1] =
 0  0
 0  0

[:, :, 2, 1] =
                     0  2.22044604925031e-16
 -2.22044604925031e-16                     0

[:, :, 1, 2] =
                    0  -2.22044604925031e-16
 2.22044604925031e-16                      0

[:, :, 2, 2] =
 0  0
 0  0

$\sigma_\mu^{\ \ AA'}\sigma^\upsilon_{\ \ AA'}=\delta^\upsilon_\mu$

In [20]:
[sum(sum(σ[μ,A,A1]*σi[υ,A,A1] for A in 1:2) for A1 in 1:2) for μ in 1:4, υ in 1:4]

4×4 Array{Sym,2}:
 -1.00000000000000                  0                  0                  0
                 0  -1.00000000000000                  0                  0
                 0                  0  -1.00000000000000                  0
                 0                  0                  0  -1.00000000000000

$\sigma_\mu^{\ \ AA'}\sigma^\mu_{\ \ BB'}-\delta^A_B\delta^{A'}_{B'}$

In [21]:
[sum(σ[μ,A,A1]*σi[μ,B,B1] for μ in 1:4)-δ[A,B]*δ[A1,B1] for A in 1:2, B in 1:2, A1 in 1:2, B1 in 1:2]

2×2×2×2 Array{Sym,4}:
[:, :, 1, 1] =
 -2.00000000000000                  0
                 0  -2.00000000000000

[:, :, 2, 1] =
 0  0
 0  0

[:, :, 1, 2] =
 0  0
 0  0

[:, :, 2, 2] =
 -2.00000000000000                  0
                 0  -2.00000000000000

In [22]:
using SymPy

@vars r θ φ real=true

# 笛卡尔坐标到球坐标的变换
X = [r*sin(θ)*cos(φ), r*sin(θ)*sin(φ), r*cos(θ)]
Q = [r, θ, φ]

# 欧氏度规及逆度规
g = sympy.eye(3) .* [1, 1, 1] 
gi = inv(g)

# 文中公式最后一项的最后一个因子
M=[diff(x, q) for x in X, q in Q]

# 球坐标下的度规  看文中改写的公式
h = simplify.(sympy.Matrix(M'*g*M))
h

3×3 Array{Sym,2}:
 1    0             0
 0  r^2             0
 0    0  r^2*sin(θ)^2

In [23]:
simplify.(sympy.Matrix(M'*g*M))

3×3 Array{Sym,2}:
 1    0             0
 0  r^2             0
 0    0  r^2*sin(θ)^2

In [24]:
[sum(M[α,μ]*g[α,β]*M[β,υ]  
        for α in  1:3, β in 1:3)  
    for μ in 1:3, υ in 1:3] .|> simplify

3×3 Array{Sym,2}:
 1    0             0
 0  r^2             0
 0    0  r^2*sin(θ)^2

In [25]:
@vars a b c d real=true

ϵ=[0 1; -1 0]
ϵi= ϵ |> inv

2×2 Array{Float64,2}:
 0.0  -1.0
 1.0   0.0

In [26]:
ϵ*ϵi

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

In [27]:
[sum(ϵi[A,B]*ϵ[B,C]  
        for B in  1:2)  
    for A in 1:2, C in 1:2] 

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0